In [ ]:
import pandas as pd
import plotly.express as px
import dash 
from dash import html,dcc
from dash.dependencies import Input,Output,State

In [ ]:
link="https://codepen.io/chriddyp/pen/bWLwgP.css"
APP=dash.Dash(external_stylesheets=[link])

In [ ]:
data=pd.read_csv("MiddleEastFreelancers_cleaning.csv")
data.head(5)

In [ ]:
data["price_per_hour"] = data["price_per_hour"].str.replace("$", '')
data["price_per_hour"] = pd.to_numeric(data["price_per_hour"])

In [ ]:
df2=data.groupby("location").agg({"price_per_hour":"sum"})
df2=df2.sort_values("price_per_hour",ascending=False)

In [ ]:
fig1=px.bar(df2 ,  y="price_per_hour" )
fig1

In [ ]:
df=pd.read_csv("MiddleEastFreelancers_cleaning.csv")
geojson = px.data.election_geojson()
candidates = df.location.unique()

In [ ]:
APP.layout=html.Div([
                       html.H1("freelancers in the middle east"),
                       html.Div([ 
                                 html.Div([dcc.Graph(figure=fig1)]),
                                 
                                          
                              ]),
                           
                       html.Div([ 
                                 html.Div([ html.P("Candidate:"),dcc.RadioItems(id='candidate', options=[{'value': x, 'label': x} for x in candidates],
                                            value=candidates[0],labelStyle={'display': 'inline-block'}),dcc.Graph(id="choropleth")
                                          ])
                                ])                        
                  ])                

In [ ]:
@APP.callback(
    Output("choropleth", "figure"), 
    [Input("country", "value")])
def display_choropleth(country):
    fig = px.choropleth(
        df, geojson=geojson, color="user_score",
        locations="location", featureidkey="properties.location",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

In [ ]:
APP.run_server()